In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(context="talk", palette="husl", color_codes=True)

from astropy.io import fits
from astropy import units as u
from astropy.convolution import Gaussian1DKernel

from specutils.spectra import Spectrum1D
from specutils.manipulation import convolution_smooth

In [2]:
# SIM_PATH = "../../../lvmdatasimulator/examples/outputs/"

# SIMPLE_SIM_NAME = "LVM_simple_example_linear_full_9000_flux.fits"
# CUSTOM_SIM_NAME = "LVM_customneb_example_linear_full_9000_flux.fits"

SIM_NAME = "DIG_bright"
EXP_TIME = 900

SIM_PATH = f"_data-sim/{SIM_NAME}/outputs"

SIMPLE_SIM_NAME = f"{SIM_NAME}_linear_full_{EXP_TIME}_flux.fits"

current_sim = SIMPLE_SIM_NAME

In [3]:
fl_rss, ef_rss = [], []
with fits.open(os.path.join(SIM_PATH, current_sim)) as f:
    wl__w = f["WAVE"].data
    for fl__w, ef__w in zip(f["TARGET"].data, f["ERR"].data):
        norm = np.abs(fl__w[fl__w.size//2])
        fl__w /= norm
        ef__w /= norm

        spectrum = Spectrum1D(fl__w*u.erg/u.s/u.cm**2/u.AA, wl__w*u.AA)
        kernel = Gaussian1DKernel(stddev=np.sqrt(2.5**2-0.5**2)/2.355/0.5)

        fl__w = convolution_smooth(spectrum, kernel).data

        fl_rss.append(fl__w)
        ef_rss.append(ef__w)

header = fits.Header()
header["CRPIX1"] = 1
header["CRVAL1"] = wl__w.min()
header["CDELT1"] = np.diff(wl__w)[0]

out_fl_name = os.path.join(SIM_PATH, current_sim.replace(".fits", ".fl.rss.fits"))
out_ef_name = os.path.join(SIM_PATH, current_sim.replace(".fits", ".ef.rss.fits"))

hdu = fits.PrimaryHDU(data=np.asarray(fl_rss), header=header)
hdu.writeto(out_fl_name, overwrite=True)

hdu = fits.PrimaryHDU(data=np.asarray(ef_rss), header=header)
hdu.writeto(out_ef_name, overwrite=True)